In [3]:
import networkx as nx
import numpy as np
import random
from queue import PriorityQueue
from copy import deepcopy

# Original graph setup
G_original = nx.DiGraph()
edges = [
    ("Attacker", "Pad", {"user": 0.6, "root": 0.6}),
    ("Attacker", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Attacker", "Host 1", {"user": 0.6, "root": 0.48}),
    ("Pad", "Host 1", {"user": 0.6, "root": 0.48}),
    ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Pad", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Pad", {"user": 0.6, "root": 0.6}),
    ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}),
    ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
    ("Host 3", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Web Server", "File Server", {"user": 0.8, "root": 0.04}),
    ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
    ("File Server", "Data Server", {"user": 0.8, "root": 0.04})
]
G_original.add_edges_from(edges)

# Node indexing (excluding Attacker for state array)
nodes = [node for node in G_original.nodes() if node != "Attacker"]
num_nodes = len(nodes)  # 7
node_to_idx = {node: idx for idx, node in enumerate(nodes)}
idx_to_node = {idx: node for node, idx in node_to_idx.items()}

# Q-learning parameters
alpha = 0.05
gamma = 0.9
epsilon_start = 0.3
epsilon_end = 0.01
epsilon_decay = 0.999
num_episodes = 1000000
max_honeypots = 2

# Initialize Q-table
Q = np.zeros((2**num_nodes, num_nodes))

# Hash state array to index
def state_to_index(state):
    return int("".join(map(str, state)), 2)

# Attacker's greedy attack with randomizer
def greedy_attack_priority_queue(graph, honeypot_nodes, goal):
    captured = {"Attacker"}
    path = ["Attacker"]
    pq = PriorityQueue()
    for neighbor in graph.successors("Attacker"):
        weight = max(graph["Attacker"][neighbor]['user'], graph["Attacker"][neighbor]['root'])
        randomizer = random.uniform(0, 1)  # Randomizer for tie-breaking
        pq.put((-weight, -randomizer, neighbor))  # Sort by -weight, -randomizer, neighbor

    while not pq.empty():
        neg_weight, neg_randomizer, to_node = pq.get()
        weight = -neg_weight
        randomizer = -neg_randomizer
        if to_node in honeypot_nodes:  # Stop at honeypot node
            path.append(to_node)
            captured.add(to_node)
            break
        if to_node not in captured:
            captured.add(to_node)
            path.append(to_node)
            if to_node == goal:
                break
            for next_node in graph.successors(to_node):
                if next_node not in captured:
                    next_weight = max(graph[to_node][next_node]['user'], graph[to_node][next_node]['root'])
                    next_randomizer = random.uniform(0, 1)  # New randomizer for each edge
                    pq.put((-next_weight, -next_randomizer, next_node))
    return path, captured

# Get possible honeypot actions (exclude Data Server and same-source nodes)
def get_possible_honeypot_actions(honeypots, source_nodes):
    return [node for node in nodes if node not in honeypots and node != "Data Server" and node not in source_nodes]

# Reward function
def get_reward(path, goal, honeypot_nodes):
    if any(node in honeypot_nodes for node in path):
        return 1  # Attacker hits a honeypot node
    if goal in path:
        return -1  # Attacker reaches Data Server
    return 0

# Choose action
def choose_action(state, honeypots, Q, epsilon, source_nodes):
    possible_actions = get_possible_honeypot_actions(honeypots, source_nodes)
    if not possible_actions:
        return None
    state_idx = state_to_index(state)
    if random.uniform(0, 1) < epsilon:
        return random.choice(possible_actions)
    else:
        q_values = [Q[state_idx][node_to_idx[node]] for node in possible_actions]
        best_action_idx = np.argmax(q_values)
        return possible_actions[best_action_idx]

# Q-learning training
initial_state = [0] * num_nodes
epsilon = epsilon_start
new_state = [0] * num_nodes

dsp=0

for episode in range(num_episodes):
    # Copy original graph
    G = deepcopy(G_original)
    honeypots = set()  # Tracks original nodes with honeypots
    honeypot_nodes = set()  # Tracks new honeypot nodes (Honeypot 1, Honeypot 2)
    source_nodes = set()  # Tracks source nodes of honeypots
    state = new_state.copy()
    new_state = [0] * num_nodes
    actions = []

    # Place honeypots
    for i in range(max_honeypots):
        action = choose_action(state, honeypots, Q, epsilon, source_nodes)
        if action is None:
            break
        honeypots.add(action)
        source_nodes.add(action)
        honeypot_name = f"Honeypot {i+1}"
        honeypot_nodes.add(honeypot_name)
        # Add temporary node and edge
        G.add_node(honeypot_name)
        G.add_edge(action, honeypot_name, user=0.8, root=0.8)
        actions.append((action, node_to_idx[action], honeypot_name))

    # Simulate attack
    attack_path, attack_captured = greedy_attack_priority_queue(G, honeypot_nodes, "Data Server")

    # Update state (only original nodes, include source if honeypot node reached)

    for node in attack_path:
        if node in nodes:  # Original node
            new_state[node_to_idx[node]] = 1
        elif node in honeypot_nodes:  # Honeypot node
            for source, _, hp_node in actions:
                if hp_node == node:
                    new_state[node_to_idx[source]] = 1
                    break

    # Calculate reward
    reward = get_reward(attack_path, "Data Server", honeypot_nodes)
    if(reward == 1):
        dsp+=1

    # Update Q-values
    state_idx = state_to_index(state)
    new_state_idx = state_to_index(new_state)
    best_next_q = min(max(np.max(Q[new_state_idx]) if get_possible_honeypot_actions(honeypots, source_nodes) else 0, -1), 1)
    for action, action_idx, _ in actions:
        Q[state_idx][action_idx] += alpha * (reward + gamma * best_next_q - Q[state_idx][action_idx])

    # Update epsilon
    epsilon = max(epsilon_end, epsilon * epsilon_decay)


    # Debug: Log episode results
    if episode % 50000 == 0:
        initial_state_idx = state_to_index(initial_state)
        q_vals = {idx_to_node[i]: Q[initial_state_idx][i] for i in range(num_nodes)}
        if episode!=0 :
            dsp/=50000
        else :
            dsp/=1.0
        print(f"Episode {episode}: Honeypots={honeypots}, Path={' -> '.join(attack_path)}, State={new_state}, Reward={reward}, Defense success probability={dsp}")
        dsp=0



# Extract optimal honeypot placement
def get_optimal_honeypots(Q):
    # state = [0] * num_nodes
    state = [0, 1, 0, 0, 0, 1, 1]
    honeypots = []
    source_nodes = set()

    for i in range(max_honeypots):
        action = choose_action(state, set(honeypots), Q, epsilon=0, source_nodes=source_nodes)
        if action is None:
            break
        honeypots.append(action)
        source_nodes.add(action)
        # Simulate attack with temporary honeypot
        G = deepcopy(G_original)
        honeypot_nodes = set()
        for j, source in enumerate(honeypots):
            hp_name = f"Honeypot {j+1}"
            G.add_node(hp_name)
            G.add_edge(source, hp_name, user=0.8, root=0.8)
            honeypot_nodes.add(hp_name)
        attack_path, _ = greedy_attack_priority_queue(G, honeypot_nodes, "Data Server")
        # Update state
        new_state = [0] * num_nodes
        for node in attack_path:
            if node in nodes:
                new_state[node_to_idx[node]] = 1
            elif node in honeypot_nodes:
                for hp_source in honeypots:
                    for j in range(len(honeypots)):
                        if G.has_edge(hp_source, f"Honeypot {j+1}") and node == f"Honeypot {j+1}":
                            new_state[node_to_idx[hp_source]] = 1
                            break
        state = new_state

    return honeypots



Episode 0: Honeypots={'Web Server', 'Pad'}, Path=Attacker -> Web Server -> File Server -> Data Server, State=[0, 1, 0, 0, 0, 1, 1], Reward=-1, Defense success probability=0.0
Episode 50000: Honeypots={'File Server', 'Web Server'}, Path=Attacker -> Web Server -> File Server -> Honeypot 1, State=[0, 1, 0, 0, 0, 1, 0], Reward=1, Defense success probability=0.45982
Episode 100000: Honeypots={'Web Server', 'File Server'}, Path=Attacker -> Web Server -> File Server -> Honeypot 2, State=[0, 1, 0, 0, 0, 1, 0], Reward=1, Defense success probability=0.4751
Episode 150000: Honeypots={'Web Server', 'Host 3'}, Path=Attacker -> Web Server -> Honeypot 2, State=[0, 1, 0, 0, 0, 0, 0], Reward=1, Defense success probability=0.469
Episode 200000: Honeypots={'Web Server', 'File Server'}, Path=Attacker -> Web Server -> File Server -> Data Server, State=[0, 1, 0, 0, 0, 1, 1], Reward=-1, Defense success probability=0.47164
Episode 250000: Honeypots={'Web Server', 'File Server'}, Path=Attacker -> Web Server ->

In [4]:

# Test and print results
optimal_honeypots = get_optimal_honeypots(Q)
print("\nFinal Results:")
print("Optimal honeypot placements:", optimal_honeypots)
# Simulate final attack
G = deepcopy(G_original)
honeypot_nodes = set()
for i, source in enumerate(optimal_honeypots):
    hp_name = f"Honeypot {i+1}"
    G.add_node(hp_name)
    G.add_edge(source, hp_name, user=0.8, root=0.8)
    honeypot_nodes.add(hp_name)
attack_path, attack_captured = greedy_attack_priority_queue(G, honeypot_nodes, "Data Server")
print("Attacker's path with honeypots:", " -> ".join(attack_path))
print("Nodes captured by attacker:", sorted(attack_captured))
print("State array after attack:", [1 if node in attack_captured and node in nodes else 0 for node in nodes])
print("Reward:", get_reward(attack_path, "Data Server", honeypot_nodes))

# Print Q-values for initial state
initial_state_idx = state_to_index([0] * num_nodes)
print("\nQ-values for initial state:")
for idx in range(num_nodes):
    print(f"- {idx_to_node[idx]}: {Q[initial_state_idx][idx]:.4f}")





#actions -> honeypot_mapping
#xoa honeypots
#nodes -> original_nodes


Final Results:
Optimal honeypot placements: ['Web Server', 'File Server']
Attacker's path with honeypots: Attacker -> Web Server -> Honeypot 1
Nodes captured by attacker: ['Attacker', 'Honeypot 1', 'Web Server']
State array after attack: [0, 1, 0, 0, 0, 0, 0]
Reward: 1

Q-values for initial state:
- Pad: -0.0500
- Web Server: -0.0500
- Host 1: 0.0000
- Host 2: 0.0000
- Host 3: 0.0000
- File Server: 0.0000
- Data Server: 0.0000
